In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import os


def load_data():
    train_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')
    test_df = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
    submission = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv')

    train_df.dropna(axis=1, inplace=True)
    y_train = np.where(train_df['target'] >= 0.5, 1, 0)

    return train_df['comment_text'], y_train, test_df['comment_text'], submission

def clf_model():
    model = LogisticRegression(random_state=42, solver='saga', max_iter=1000, n_jobs=-1)
    return model

def vectorizer():
    return TfidfVectorizer()


train_data, train_label, test_data, submission = load_data()

vect = vectorizer()
x = vect.fit_transform(train_data)
test_x = vect.transform(test_data)

x_train, x_validation, y_train, y_validation = train_test_split(x, train_label, test_size=0.2, random_state=42, stratify=train_label)


clf = clf_model()
clf.fit(x_train, y_train)

y_predict = clf.predict(x_validation)
print(classification_report(y_validation, y_predict))

predictions = clf.predict_proba(test_x)[:, 1]
submission['prediction'] = predictions
submission.to_csv('submission.csv', index=False)
# cross_val_score(clf, x_train, )

